This notebook was originally written by Pat Gunn, Eftychios Pnevmatikakis, Johannes Friedrich, and Andrea Giovannucci from the CaImAn project:  https://github.com/flatironinstitute/CaImAn/blob/master/demos/notebooks/demo_pipeline_cnmfE.ipynb

It was later edited by Oliver Barnstedt for the LINdoscope 2021 course: http://www.lindoscope.com

The full CaImAn documentation can be found here: 
https://caiman.readthedocs.io/en/master/core_functions.html

## Pipeline for microendoscopic data processing in CaImAn using the CNMF-E algorithm
This demo presents a complete pipeline for processing microendoscopic data using CaImAn. It includes:
- Preprocessing of data acquired with Inscopix miniscopes
- Motion Correction using the NoRMCorre algorithm
- Source extraction using the CNMF-E algorithm
- Deconvolution using the OASIS algorithm

Some basic visualization is also included. The demo illustrates how to `params`, `MoctionCorrection` and `cnmf` object for processing 1p microendoscopic data. For more information see the companion CaImAn paper.

In [ ]:
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    get_ipython().magic(u'matplotlib qt')
except:
    pass

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
import os
import glob
from tqdm import tqdm

import logging
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.DEBUG)

import caiman as cm
from caiman.base.movies import load
from caiman.summary_images import local_correlations
from caiman.source_extraction import cnmf
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr, nb_inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.base.rois import extractROIsFromPCAICA
import cv2
import napari

try:
    cv2.setNumThreads(0)
except:
    pass
import bokeh.plotting as bpl
import holoviews as hv
bpl.output_notebook()
hv.notebook_extension('bokeh')
from interactivecrop.interactivecrop import main as crop

### Preprocess video files
The miniscope software saves the raw calcium imaging videos as several large TIFFs. These should be cropped, spatially downsampled, and then concatenated before they can be processed.

In [ ]:
# Enter file directory
filedir = '/Users/Oliver/LINdoscope2021_files/1P_invitro'

# Create 'preprocessed' subfolder
new_dir = os.path.join(filedir, 'preprocessed')
if not os.path.exists(new_dir):
    os.makedirs(new_dir)

samplesize = 300  # how many frames to load as sample

fnames_raw = sorted(glob.glob(os.path.join(filedir, '*.avi*')))  # finds all TIFF files in filedir folder
print(fnames_raw)
sample = load(fnames_raw[0], subindices=(range(samplesize)))  # load first couple of frames

In [ ]:
# EXERCISE: print dimensions of video sample



In [ ]:
# EXERCISE: Use Matplotlib's IMSHOW function to plot the first frame. How many cells can you see by eye?



In [ ]:
# use CaImAn's local_correlations function to create a correlation image
cn = local_correlations(sample.transpose(1,2,0))  # needs to be transposed to XYT

In [ ]:
# to facilitate processing, we can use the correlation image to crop out XY parts of the video without cells
cn_lut = cv2.applyColorMap(np.uint8(cn*255), cv2.COLORMAP_RAINBOW)  # convert correlation image to RGB
crop([cn_lut])  # interactive cropping

In [ ]:
# confirm cropping
cropping = (60, 0, 647, 480)  # manually copy from interactive cropping results above!  #Inscopix sample: (97, 0, 982, 800)
cropped_cn = cn[cropping[1]:cropping[1]+cropping[3],cropping[0]:cropping[0]+cropping[2]]
plt.imshow(cropped_cn);

In [ ]:
# CROP AND DOWNSAMPLE RAW VIDEO (TAKES SOME TIME)
sds_ratio = .5  # spatial downsampling  #Isx: .25
tds_ratio = .5  # temporal downsampling

fnames = [os.path.join(new_dir, os.path.basename(f)[:-4]+'_ds.tif') for f in fnames_raw]  # downsampled file names

for i, moviepath in tqdm(enumerate(fnames_raw)):
    movie = cm.load(moviepath)

    # CROP
    movie_cropped = movie[:, cropping[1]:cropping[1]+cropping[3],cropping[0]:cropping[0]+cropping[2]]

    # RESIZE
    movie_resized = movie_cropped.resize(sds_ratio, sds_ratio, tds_ratio)  # resizing
    movie_resized.save(fnames[i])

    # CONCATENATE if several files exist
    if i==0:
        movies_concat = movie_resized.copy()
    else:
        movies_concat = cm.concatenate([movies_concat, movie_resized], axis=0)
    del movie, movie_cropped, movie_resized  # delete objects to save memory

# save (concatenated) movie
if 'movies_concat' in locals() and i>0:
    fname = os.path.join(new_dir, os.path.basename(moviepath)[:-4]+'_concatenated.tif')
    if not os.path.exists(fname):
        movies_concat.save(fname)
else:
    fname = fnames[0]

In [ ]:
# if already downsampled in FIJI:
fname = '/Users/Oliver/LINdoscope2021_files/1P_invitro/preprocessed/Untitled_3_MMStack_Default.tif'
fnames = ['/Users/Oliver/LINdoscope2021_files/1P_invitro/preprocessed/Untitled_3_MMStack_Default_ds.tif']
movies_concat = cm.load(fnames)

In [ ]:
# save as memmap
bord_px = 0
fname_new = cm.save_memmap(fnames, base_name='memmap_', order='C', border_to_0=bord_px)

# load memory mappable file
Yr, dims, T = cm.load_memmap(fname_new)
motion_corrected = Yr.T.reshape((T,) + dims, order='F')

In [ ]:
%%capture
# inspect (first) downsampled file
if 'movies_concat' not in locals():
    movies_concat = cm.load(fname)
viewer = napari.view_image(movies_concat, colormap='viridis');

### Visualise spatial filtering

In [ ]:
gSig_filt = (3, 3)       # size of high pass spatial filtering, used in 1p data  #Isx(2,2)
ksize = tuple([(3 * i) // 2 * 2 + 1 for i in gSig_filt])
ker = cv2.getGaussianKernel(ksize[0], gSig_filt[0])
ker2D = ker.dot(ker.T)
nz = np.nonzero(ker2D >= ker2D[:, 0].max())
zz = np.nonzero(ker2D < ker2D[:, 0].max())
ker2D[nz] -= ker2D[nz].mean()
ker2D[zz] = 0
movies_hpfilt = cm.movie(np.array([cv2.filter2D(np.array(img, dtype=np.float32), -1, ker2D, borderType=cv2.BORDER_REFLECT) for img in movies_concat]))  

In [ ]:
# normalise filtered video and view side-by-side in NAPARI
movies_hpfilt_norm = movies_hpfilt + float(np.min(movies_hpfilt))
scale_factor = float(np.max(movies_concat)-np.min(movies_concat)) / float(np.max(movies_hpfilt)-np.min(movies_hpfilt))
movies_hpfilt_norm = movies_hpfilt_norm * scale_factor + float(np.min(movies_concat))
view_spatial_filtering = np.concatenate([movies_concat, movies_hpfilt_norm], 2)
viewer = napari.view_image(view_spatial_filtering, colormap='viridis');

In [ ]:
# save video
movies_hpfilt_norm.save(fname[:-4]+'_hpfilt.tif')

### Setup a cluster
To enable parallel processing a (local) cluster needs to be set up. This is done with a cell below. The variable `backend` determines the type of cluster used. The default value `'local'` uses the multiprocessing package. The `ipyparallel` option is also available. More information on these choices can be found [here](https://github.com/flatironinstitute/CaImAn/blob/master/CLUSTER.md). The resulting variable `dview` expresses the cluster option. If you use `dview=dview` in the downstream analysis then parallel processing will be used. If you use `dview=None` then no parallel processing will be employed.

In [ ]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

### Setup some parameters
We first set some parameters related to the data and motion correction and create a `params` object. We'll modify this object with additional settings later on. You can also set all the parameters at once as demonstrated in the `demo_pipeline.ipynb` notebook.

In [ ]:
# dataset dependent parameters
frate = 10                      # movie frame rate
decay_time = 0.8                 # length of a typical transient in seconds; optimal for GCaMP6f (fast) 0.3

# motion correction parameters
motion_correct = True    # flag for performing motion correction
pw_rigid = False         # flag for performing piecewise-rigid motion correction (otherwise just rigid)
gSig_filt = (3, 3)       # size of high pass spatial filtering, used in 1p data
max_shifts = (20, 20)    # maximum allowed rigid shift
border_nan = 'copy'      # replicate values along the boundaries

mc_dict = {
    'fnames': fnames,
    'fr': frate,
    'decay_time': decay_time,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'gSig_filt': gSig_filt,
    'border_nan': border_nan
}

opts = params.CNMFParams(params_dict=mc_dict)

In [ ]:
# MOTION CORRECTION (takes some time)
mc = MotionCorrect(fname, dview=dview, **opts.get_group('motion'))
mc.motion_correct(save_movie=True)  # motion corrects and saves memory mapped file in order 'F'


In [ ]:
# plotting
bord_px = np.ceil(np.max(np.abs(mc.shifts_rig))).astype(np.int)
plt.subplot(1, 2, 1); plt.imshow(mc.total_template_rig)  # % plot template
plt.subplot(1, 2, 2); plt.plot(mc.shifts_rig)  # % plot rigid shifts
plt.legend(['x shifts', 'y shifts'])
plt.xlabel('frames')
plt.ylabel('pixels')

# saving memory mapped file in order 'C'
bord_px = 0 if border_nan is 'copy' else bord_px
fname_mc = mc.fname_tot_rig
fname_new = cm.save_memmap(fname_mc, base_name='memmap_', order='C', border_to_0=bord_px)

In [ ]:
# remove movies from workspace to save memory
del movies_concat, movies_hpfilt_norm, movies_hpfilt_mc, vis_mc_nofilter, vis_mc_filtered, vis_mc

In [ ]:
# display motion correction
fname_mc = mc.fname_tot_rig
bord_px = np.ceil(np.max(np.abs(mc.shifts_rig))).astype(np.int)
plt.subplot(1, 2, 1); plt.imshow(mc.total_template_rig)  # % plot template
plt.subplot(1, 2, 2); plt.plot(mc.shifts_rig)  # % plot rigid shifts
plt.legend(['x shifts', 'y shifts'])
plt.xlabel('frames')
plt.ylabel('pixels')

bord_px = 0 if border_nan is 'copy' else bord_px
fname_new = cm.save_memmap(fname_mc, base_name='memmap_', order='C',
                           border_to_0=bord_px)

# load memory mappable file
Yr, dims, T = cm.load_memmap(fname_new)
motion_corrected = Yr.T.reshape((T,) + dims, order='F')

# apply motion correction to filtered movie
movies_hpfilt_mc = mc.apply_shifts_movie(fname[:-4]+'_hpfilt.tif')  # apply motion correction to filtered video

In [ ]:
def run_pca_ica(fnames):
    m = cm.load(fnames)
    
    # run pca-ica
    output, _ = m.IPCA_stICA(componentsICA=15, mu=0.05)
    masks = output.copy()
    masks = np.array(extractROIsFromPCAICA(masks)[0])
    masks = masks / np.linalg.norm(masks, ord='fro', axis=(1,2))[:, np.newaxis, np.newaxis]
    spatial = masks.copy()
    
    plt.imshow(spatial.sum(0));plt.show()

    # from masks recover signal
    temporal = m.extract_traces_from_masks(masks)
    temporal = -signal_filter(temporal.T, freq=15, fr=400).T
    
    result = {'spatial':spatial, 'temporal':temporal}
    save_path = os.path.join(os.path.split(fnames)[0], 'pca-ica', f'pca-ica_{os.path.split(fnames)[1][:-5]}')
    np.save(save_path, result)

### Parameter setting for CNMF-E
We now define some parameters for the source extraction step using the CNMF-E algorithm. 
We construct a new dictionary and use this to modify the *existing* `params` object,

In [ ]:
# parameters for source extraction and deconvolution
p = 1               # order of the autoregressive system
K = 200            # upper bound on number of components per patch, in general None  # 100
gSig = (3, 3)       # gaussian width of a 2D gaussian kernel, which approximates a neuron  #(3,3)
gSiz = None     # average diameter of a neuron, in general 4*gSig+1
Ain = None          # possibility to seed with predetermined binary masks
merge_thr = .5      # merging threshold, max correlation allowed
rf = 40             # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
stride_cnmf = 20    # amount of overlap between the patches in pixels
#                     (keep it at least as large as gSiz, i.e 4 times the neuron size gSig)
tsub = 2            # downsampling factor in time for initialization,
#                     increase if you have memory problems
ssub = 1            # downsampling factor in space for initialization,
#                     increase if you have memory problems
#                     you can pass them here as boolean vectors
low_rank_background = None  # None leaves background of each patch intact,
#                     True performs global low-rank approximation if gnb>0
gnb = 0             # number of background components (rank) if positive,
#                     else exact ring model with following settings
#                         gnb= 0: Return background as b and W
#                         gnb=-1: Return full rank background B
#                         gnb<-1: Don't return background
nb_patch = 0        # number of background components (rank) per patch if gnb>0,
#                     else it is set automatically
min_corr = .8       # min peak value from correlation image
min_pnr = 10        # min peak to noise ratio from PNR image
ssub_B = 2          # additional downsampling factor in space for background
ring_size_factor = 1.4  # radius of ring is gSiz*ring_size_factor

opts.change_params(params_dict={'method_init': 'corr_pnr',  # use this for 1 photon
                                'K': K,
                                'gSig': gSig,
                                'gSiz': gSiz,
                                'merge_thr': merge_thr,
                                'p': p,
                                'tsub': tsub,
                                'ssub': ssub,
                                'rf': rf,
                                'stride': stride_cnmf,
                                'only_init': True,    # set it to True to run CNMF-E
                                'nb': gnb,
                                'nb_patch': nb_patch,
                                'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
                                'low_rank_background': low_rank_background,
                                'update_background_components': True,  # sometimes setting to False improve the results
                                'min_corr': min_corr,
                                'min_pnr': min_pnr,
                                'normalize_init': False,               # just leave as is
                                'center_psf': True,                    # leave as is for 1 photon
                                'ssub_B': ssub_B,
                                'ring_size_factor': ring_size_factor,
                                'del_duplicates': True,                # whether to remove duplicates from initialization
                                'border_pix': None})                # number of pixels to not consider in the borders)

### Inspect summary images and set parameters
Check the optimal values of `min_corr` and `min_pnr` by moving slider in the figure that pops up. You can modify them in the `params` object. 
Note that computing the correlation pnr image can be computationally and memory demanding for large datasets. In this case you can compute
only on a subset of the data (the results will not change). You can do that by changing `images[::1]` to `images[::5]` or something similar.
This will compute the correlation pnr image

In [ ]:
# compute some summary images (correlation and peak to noise)
cn_filter, pnr = cm.summary_images.correlation_pnr(motion_corrected[::1], gSig=gSig[0], swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile
# inspect the summary images and set the parameters
nb_inspect_correlation_pnr(cn_filter, pnr)

You can inspect the correlation and PNR images to select the threshold values for `min_corr` and `min_pnr`. The algorithm will look for components only in places where these value are above the specified thresholds. You can adjust the dynamic range in the plots shown above by choosing the selection tool (third button from the left) and selecting the desired region in the histogram plots on the right of each panel.

In [ ]:
# print parameters set above, modify them if necessary based on summary images
print(min_corr) # min correlation of peak (from correlation image)
print(min_pnr)  # min peak to noise ratio

### Run the CNMF-E algorithm (takes some time)

In [ ]:
plt.imshow(motion_corrected[500,:,:])

In [ ]:
cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=Ain, params=opts)
cnm.fit(motion_corrected)
cnm.save(fnames[0][:-4]+'_cnm.hdf5')  # save file


## Component Evaluation

The processing in patches creates several spurious components. These are filtered out by evaluating each component using three different criteria:

- the shape of each component must be correlated with the data at the corresponding location within the FOV
- a minimum peak SNR is required over the length of a transient

After setting some parameters we again modify the existing `params` object.

In [ ]:
#%% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier

min_SNR = 5            # adaptive way to set threshold on the transient size
r_values_min = 0.2    # threshold on space consistency (if you lower more components
#                        will be accepted, potentially with worse quality)
cnm.params.set('quality', {'min_SNR': min_SNR,
                           'rval_thr': r_values_min,
                           'use_cnn': False})
cnm.estimates.evaluate_components(motion_corrected, cnm.params, dview=dview)

print(' ***** ')
print('Number of total components: ', len(cnm.estimates.C))
print('Number of accepted components: ', len(cnm.estimates.idx_components))

In [ ]:
sum(cnm.estimates.SNR_comp<5)

In [ ]:
cnm.estimates.r_values>0.8

### Do some plotting

In [ ]:
#%% plot contour plots of accepted and rejected components
cnm.estimates.plot_contours_nb(img=cn_filter, idx=cnm.estimates.idx_components)

View traces of accepted and rejected components. Note that if you get data rate error you can start Jupyter notebooks using:
'jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10'

In [ ]:
# accepted components
cnm.estimates.hv_view_components(img=cn_filter, idx=cnm.estimates.idx_components,
                                denoised_color='red', cmap='gray')

In [ ]:
# rejected components
cnm.estimates.hv_view_components(img=cn_filter, idx=cnm.estimates.idx_components_bad,
                                denoised_color='red', cmap='gray')

### Stop cluster

In [ ]:
cm.stop_server(dview=dview)

### Some instructive movies
Play the reconstructed movie alongside the original movie and the (amplified) residual

In [ ]:
# with background 
cnm.estimates.play_movie(motion_corrected, q_max=99.5, magnification=2, include_bck=True, gain_res=10, bpx=bord_px);

In [ ]:
# without background
cnm.estimates.play_movie(motion_corrected, q_max=99.9, magnification=2, include_bck=False, gain_res=4, bpx=bord_px);